<p style="color:Blue"><font size="6">Stochastic Diffusion Search</font></p>

<p ><font size="4">An approach to solve the Curse of Dimensionality in Data Science using SDS Feature Selection</font></p> 

<p ><font size="3">This is a project done for the Optimization course of The Informatics Department at AUTh.</font></p>


<p ><font size="3"> The theory behind the algorithm is originally published <a href="https://dl.acm.org/citation.cfm?id=3079193">here</a>.</font></p>

### Importing necessary libraries

In [25]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
import time

### Creating our estimators

In [26]:
logReg=LogisticRegression(C=2)
decClf=DecisionTreeClassifier(max_depth=5, min_samples_split=4)
rfc=RandomForestClassifier()

In [27]:
estimators=[rfc,logReg,decClf] 

_____
## 1) Initialization Phase - Hypothesis Selection 
_____

### Function to randomly select an agents hypothesis.

In [28]:
'''
Below function returns an agent, which is hypothesis, and its corresponding binary array.
1 indicates inclusion of corresponding feature and 0 indicates exclusion of the feature.
lowerLim indicates minimum number of features, whereas; upperLim indicated max no of features to beincluded in an agent.
'''
def agent(arryX,lowerLim,upperLim):
        if lowerLim<0 or upperLim>arryX.shape[1]:
            print('recall function with appropriate limits')
        else:
            randomNoFeatures=np.random.randint(lowerLim,upperLim,size=1)[0] #generating a random number
            zeroArry=np.zeros(arryX.shape[1]-randomNoFeatures, dtype='int') #zero array 
            oneArry=np.ones(randomNoFeatures, dtype='int')   #one array
            fArry=np.concatenate((zeroArry,oneArry), axis=0) #concatinating zero and one array
            np.random.shuffle(fArry) #shuffling fArray
            fIndex=np.where(fArry==1)[0]
            agentArry=arryX[:,fIndex] #generating feature subset from origanal dataset
            return fArry,agentArry #returns array of 0s and 1s of features selected, and the new agent's dataset

### Function to initialize all agents' hypotheses.

In [29]:
'''
Below function generates required number of agents that are to be deployed on search space. 
All the agents and corresponding binary feature array are stored and returned as a list.
'''
def agentsInitiation(arryX,numAgents,lowerLim,upperLim):
        agents=[]
        agentFIndex=[]
        agentStatus=['active']*numAgents #create a status list of 'active' agents
        for i in range(0,numAgents):
            fArry,agentArry=agent(arryX,lowerLim,upperLim) #generating a single agent
            agentFIndex.append(fArry) #appending its binary feature array to agentFIndex (list to store each agent's 0-1 array)
            agents.append(agentArry) #appending the agent to the agents list (list to store each agent's dataset)
        return agents,agentFIndex,agentStatus #return the 2 lists above, and a list of 'active' statuses

### Function to calculate the score of an agent's hypothesis 

<p style="color:Red"><font size="2">TODO: Create actual ensemble - This is just averaging scores</font></p>

In [30]:
'''
'Score' function fits each model to the agent's training data and then evaluates the score on test data. 
The output is the average score of three estimators. We calculate the average so we can have an ensemble of model predictions,
and thus not be biased toward one specific model's predictions.
'''
def score(estimators,arryX,arrY):
        X_train,X_test,y_train,y_test=train_test_split(arryX,arrY,random_state=0)
        scores=[]
        for i in range(len(estimators)):
            estimators[i].fit(X_train,y_train) #fitting the ith estimator to the training data of an agent
            scores.append(estimators[i].score(X_test,y_test)) #evaluating the score on the test data
        return sum(scores)/len(scores)

### Function to calculate the score of all agents and save them

In [31]:
#below function calculates score for each agents and appends the score to the agentScores list
def agentClfscores(estimators,agents,arrY):
    agentScores=[]
    for agent in agents:
        agentscore=score(estimators,agent,arrY)
        agentScores.append(agentScores)
    return agentScores #returns a list that captures agents' scores

_____
## 2) Testing Phase - Turn agents into active ones or inactive ones based on partial evaluation

## 3) Diffusion Phase - Information diffusion between agents
____

### Function that carries out the testing and diffusion phase

In [32]:
'''
Below function carries out test and diffusion phase among the agents initialized above. 
The function returns agents and their corresponding scores, binary feature arrays, and staus after numIterations.
'''

def SDSFS(arryX,arrY,estimators,numIterations,numAgents,lowerLim,upperLim):
    agents,agentFIndex,agentStatus=agentsInitiation(arryX,numAgents,lowerLim,upperLim) #agents' datasets, 0-1 arrays, active' status
    agentScores=agentClfscores(estimators,agents,arrY) # list of agents' scores
    niters=0
    while niters<numIterations: # um of iterations that testing and diffusion will happen
        print("Iter: " + str(niters))
        #Testing phase
        for i in range(len(agents)):
            rndmId=np.random.randint(len(agents),size=1)[0]
            if agentScores[i]>agentScores[rndmId]: # if ith agents has better score than a random agent, then become 'active' or 'happy' 
                agentStatus[i]='active' 
                
            else:                                  # else if ith agent has worse score than a random agent, become 'inactive' or 'unhappy'
                agentStatus[i]='inactive'
                
        #Diffusion phase    
        for i in range(len(agents)):
# ========= if ith agent is 'unhappy' =========
            if agentStatus[i]=='inactive': 
                rndmId2=np.random.randint(len(agents),size=1)[0]
# ============= if random agent is 'happy' =============
                if agentStatus[rndmId2]=='active': 
                    oneIds=np.where(agentFIndex[rndmId2]==1)[0] # find where are the random agent's 1s
                    #print(oneIds)
                    zeroIds=np.where(agentFIndex[rndmId2]==0)[0] # find where are the random agent's 0s
                    rndmId3=np.random.randint(len(oneIds), size=1) # create n1 random number in range [0,numOf1s]
                    rndmId4=np.random.randint(len(zeroIds), size=1) # create n0 random number in range [0,numOf0s]
                    oneZeroId=oneIds[rndmId3] # choose a 1-feature at random (from the random agent's 1s)
                    zeroOneId=zeroIds[rndmId4] # choose a 0-feature at random (from the random agent's 0s)
                    agentFIndex[i]=agentFIndex[rndmId2].copy() # copy the random agent's 0-1 array
                    agentFIndex[i][oneZeroId]=0 # change this specific 1-feature to 0 (evolution?)
                    agentFIndex[i][zeroOneId]=1 # change this specific 0-feature to 1 (evolution?)
                    fIndex2=np.where(agentFIndex[i]==1)[0] # find where are the ith agent's 1s
                    agents[i]=X[:,fIndex2] # create new dataset for the ith agent
                    agentScores[i]=score(estimators,agents[i],arrY) # compute new score 
# ============= else if random agent is 'unhappy' =============
                else:            
                    agentFIndex[i],agents[i]=agent(arryX,lowerLim,upperLim) # try again with different random combination
                    agentScores[i]=score(estimators,agents[i],arrY) # compute new score  
# ========= else if ith agent is 'happy' ========= 
            else:                           
                rndmId5=np.random.randint(len(agents),size=1)[0]
                # if random agent == ith agent
                if agentStatus[rndmId5]=='active' and (agentFIndex[i]==agentFIndex[rndmId5]).all():
                    agentStatus[i]='inactive' # make ith agent 'inactive' or 'unhappy' (escape local minimal)
                    agentFIndex[i],agents[i]=agent(arryX,lowerLim,upperLim) # try again with different random combination
                    agentScores[i]=score(estimators,agents[i],arrY)
        niters+=1
    return agents,agentFIndex,agentStatus,agentScores
    

<p style="color:Green"><font size="5">Experiments</font></p>

<p style="color:Red"><font size="4">1) Sonar Signals</font></p>

The dataset contains 111 patterns obtained by bouncing sonar signals off a metal cylinder at various angles and under various conditions. 
It contains 97 patterns obtained from rocks under similar conditions. 
The transmitted sonar signal is a frequency-modulated chirp, rising in frequency. 
The data set contains signals obtained from a variety of different aspect angles, spanning 90 degrees for the cylinder and 180 degrees for the rock. 

More info on the dataset can be found <a href="https://www.openml.org/d/40">here</a>.</font></p>

In [213]:
df=pd.read_csv('./Sonar.csv')

In [214]:
df.shape #the dataset has 60 features and one class column
rows = df.shape[0]

In [215]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,1


In [216]:
df['Class'].value_counts() #the dataset has 111 class 0 instances and 97 class 1 instances

0    111
1     97
Name: Class, dtype: int64

## Drop missing values

In [217]:
df.dropna(inplace=True) #deleting any rows with missing values

In [218]:
df.isnull().any().isnull().any() #confirming there are no missing values in the dataframe

False

## Create X and y set

In [219]:
X=np.array(df[list(df.columns[:-1])])

y=df['Class']

original_cols = X.shape[1]

In [220]:
X_train,X_test,y_train,y_test=train_test_split(X,y, random_state=0) 

## Logistic Regression

In [221]:
start = time.time()

logReg.fit(X_train,y_train) #fitting Logistic Regression on Original Dataset

end = time.time()
print('Time elapsed: :' + str(end - start) + ' seconds')

logReg_score=logReg.score(X_test,y_test) #Test score
print('Losgistic Regression Score: ' + str(logReg_score))

Time elapsed: :0.035976409912109375 seconds
Losgistic Regression Score: 0.7692307692307693


## Random Forest

In [294]:
start = time.time()

rfc.fit(X_train,y_train) #fitting Random Forest to the original dataset

end = time.time()
print('Time elapsed: :' + str(end - start) + ' seconds')

rfc_score=rfc.score(X_test,y_test)
print('Random Forest Score: ' + str(rfc_score))

Time elapsed: :0.14693069458007812 seconds
Random Forest Score: 0.8461538461538461


## Decision Tree

In [223]:
start = time.time()

decClf.fit(X_train,y_train) #fitting decision tree classifer to the original dataset

end = time.time()
print('Time elapsed: :' + str(end - start) + ' seconds')

decClf_score=decClf.score(X_test,y_test)
print('Decision Tree Score: ' + str(decClf_score))

Time elapsed: :0.006985902786254883 seconds
Decision Tree Score: 0.75


Now we will try to run SDSFS algorithm to check if it can improve the accuracy of the classifiers

In [467]:
start = time.time()

agents,agentFIndex,agentStatus,agentScores=SDSFS(X,y,estimators,20,15,2,60)

end = time.time()
print(end - start)

Iter: 0
Iter: 1
Iter: 2
Iter: 3
Iter: 4
Iter: 5
Iter: 6
Iter: 7
Iter: 8
Iter: 9
Iter: 10
Iter: 11
Iter: 12
Iter: 13
Iter: 14
Iter: 15
Iter: 16
Iter: 17
Iter: 18
Iter: 19
32.56635761260986


In [468]:
max(agentScores) #one of the agents achieved an accuracy of 86.5 percent

0.8653846153846153

In [469]:
best = np.argmax(np.array(agentScores)) 

In [470]:
cols = agents[best].shape[1] 
print(cols)

36


Now we will train the models on this agent instead of original dataset

In [471]:
X_train,X_test,y_train,y_test=train_test_split(agents[best],y, random_state=0) 

In [472]:
logReg.fit(X_train,y_train)

LogisticRegression(C=2)

In [473]:
logReg_score2=logReg.score(X_test,y_test)
print(logReg_score2)

0.8461538461538461


In [474]:
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [475]:
rfc_score2=rfc.score(X_test,y_test) 
print(rfc_score2)

0.8846153846153846


In [496]:
decClf.fit(X_train,y_train) 

DecisionTreeClassifier(max_depth=5, min_samples_split=4)

In [497]:
decClf_score2=decClf.score(X_test,y_test) 
print(decClf_score2)

0.7692307692307693


In [498]:
final_result=pd.DataFrame([[logReg_score,rfc_score,decClf_score],
                           [logReg_score2,rfc_score2,decClf_score2]],
                          columns=['Logistic Regression','Random Forest','Decision Tree'], 
                          index=['original dataset ' + str(rows) + ' rows, ' + str(original_cols) + ' cols',
                                 'SDSFS subset ' + str(rows) + ' rows, ' + str(cols) + ' cols'])

In [499]:
final_result

,Logistic Regression,Random Forest,Decision Tree
"original dataset 208 rows, 60 cols",0.769231,0.846154,0.750000
"SDSFS subset 208 rows, 36 cols",0.846154,0.884615,0.769231


Accuracy improved for all the models with the subset obtained from SDSFS algorithm. I'm still trying to improve the algorithm to get more consistent and reliable results.

# --- We don't need more experiments here ---

## Create X and y set

In [174]:
end = 180
start = 50

X=np.array(df[list(df.columns[:-1])][start:end]) 

y=df['Class'][start:end]

original_cols = X.shape[1]

cols=end-start

In [175]:
X.shape

(130, 60)

In [176]:
X_train,X_test,y_train,y_test=train_test_split(X,y, random_state=0) 

## Logistic Regression

In [177]:
start = time.time()

logReg.fit(X_train,y_train) #fitting Logistic Regression on Original Dataset

end = time.time()
print('Time elapsed: ' + str(round(end - start,2)) + ' seconds')

logReg_score3=logReg.score(X_test,y_test) #Test score
print('Losgistic Regression Score: ' + str(logReg_score3))

Time elapsed: 0.01 seconds
Losgistic Regression Score: 0.9393939393939394


## Random Forest

In [178]:
start = time.time()

rfc.fit(X_train,y_train) #fitting Random Forest to the original dataset

end = time.time()
print('Time elapsed: ' + str(end - start) + ' seconds')

rfc_score3=rfc.score(X_test,y_test)
print('Random Forest Score: ' + str(rfc_score3))

Time elapsed: 0.14090204238891602 seconds
Random Forest Score: 0.9393939393939394


## Decision Tree

In [179]:
start = time.time()

decClf.fit(X_train,y_train) #fitting decision tree classifer to the original dataset

end = time.time()
print('Time elapsed: ' + str(end - start) + ' seconds')

decClf_score3=decClf.score(X_test,y_test)
print('Decision Tree Score: ' + str(decClf_score3))

Time elapsed: 0.004979848861694336 seconds
Decision Tree Score: 0.8181818181818182


In [189]:
start = time.time()

agents,agentFIndex,agentStatus,agentScores=SDSFS(X,y,estimators,10,15,10,20)

end = time.time()
print(end - start)

Iter: 0
Iter: 1
Iter: 2
Iter: 3
Iter: 4
Iter: 5
Iter: 6
Iter: 7
Iter: 8
Iter: 9
15.128660440444946


In [190]:
max(agentScores) 

0.9292929292929294

In [191]:
best = np.argmax(np.array(agentScores)) 

cols = agents[best].shape[1] 
print(cols)

15


In [192]:
X_train,X_test,y_train,y_test=train_test_split(agents[best],y, random_state=0) 

In [193]:
logReg.fit(X_train,y_train)
logReg_score4=logReg.score(X_test,y_test) 
print(logReg_score4)

0.8484848484848485


In [194]:
rfc.fit(X_train,y_train)
rfc_score4=rfc.score(X_test,y_test) 
print(rfc_score4)

0.9393939393939394


In [195]:
decClf.fit(X_train,y_train) 
decClf_score4=decClf.score(X_test,y_test) 
print(decClf_score4)

0.9393939393939394


In [196]:
final_result=pd.DataFrame([[logReg_score3,rfc_score3,decClf_score3],
                           [logReg_score4,rfc_score4,decClf_score4]],
                          columns=['Logistic Regression','Random Forest','Decision Tree'], 
                          index=['original dataset ' + str(rows) + ' rows, ' + str(original_cols) + ' cols',
                                 'SDSFS subset ' + str(rows) + ' rows, ' + str(cols) + ' cols'])

In [197]:
final_result

,Logistic Regression,Random Forest,Decision Tree
"original dataset 150 rows, 60 cols",0.939394,0.939394,0.818182
"SDSFS subset 150 rows, 15 cols",0.848485,0.939394,0.939394
